### Directories

- mirror:  Old mirrored PDF files
- mirror-csv:  New (2017) mirrored CSV files

In [6]:
import glob, os, subprocess, tempfile, time, urllib2

In [2]:
source_url = "http://www.achd.net/airqual/DailySummary.PDF"
dest_dir = "mirror-csv"
tmp_dir = "tmp"

print "Mirror directory is %s" % os.path.abspath(dest_dir)

Mirror directory is /usr4/web/data.cmucreatelab.org/mirror-achd-air-quality-reports-esdr/mirror-csv


In [3]:
def subprocess_check(*args, **kwargs):
    if len(args) == 1 and type(args[0]) == str:
        kwargs['shell'] = True
    p = subprocess.Popen(*args,  
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         **kwargs)
    (out, err) = p.communicate()
    ret = p.wait()
    if ret != 0:
        if out and out[-1] != '\n':
            out += '\n'
        if err and err[-1] != '\n':
            err += '\n'
        raise Exception(
            ('Call to subprocess_check failed with return code {ret}\n'
             'Standard error:\n{err}'
             'Standard out:\n{out}').format(**locals()))
    if err != '' and out != '' and err[-1] != '\n':
        err += '\n'
    return err + out

In [8]:
def now():
    return time.strftime('%Y-%m-%d-%H:%M:%S%z')

def find_most_recent_path(dest_dir):
    mirrored_files = sorted(glob.glob(dest_dir + "/AirQualityDataSummary-????-??-??-??:??:??*.csv"))
    if len(mirrored_files) == 0:
        return None
    return mirrored_files[-1]

def mirror_achd_sftp(dest_dir, tmp_dir):
    try:
        os.mkdir(dest_dir)
    except OSError:
        pass
    try:
        os.mkdir(tmp_dir)
    except OSError:
        pass

    tmp_filename = tempfile.NamedTemporaryFile(dir=tmp_dir, delete=False).name

    src = 'pdille@moveit.alleghenycounty.us:/WPRDC/Health\ Department/Air\ Quality\ Daily\ Summary.CSV'
    cmd = ['sshpass',
       '-f%s/achd-ftp-passwd-do-not-check-in.txt' % os.getcwd(),
       'sftp',
       src,
       tmp_filename]
    print ' '.join(cmd)
    
    subprocess_check(cmd)
    data = open(tmp_filename).read()
    print '%s: Fetched %d bytes from %s to %s\n' % (now(), len(data), src, tmp_filename)
              
    most_recent_path = find_most_recent_path(dest_dir)
              
    if most_recent_path and open(most_recent_path).read() == data:
        print "%s: Not recording %d bytes read from %s because identical to previous file %s\n" % (now(), len(data), src, most_recent_path)
        os.unlink(tmp_filename)
    else:
        dest = "%s/AirQualityDataSummary-%s.csv" % (dest_dir, now())
        os.chmod(tmp_filename, 0644)
        os.rename(tmp_filename, dest)
        print "%s: Stored %d bytes read from %s at path %s\n" % (now(), len(data), src, dest)
    

In [9]:
mirror_achd_sftp(dest_dir, tmp_dir)

sshpass -f/usr4/web/data.cmucreatelab.org/mirror-achd-air-quality-reports-esdr/achd-ftp-passwd-do-not-check-in.txt sftp pdille@moveit.alleghenycounty.us:/WPRDC/Health\ Department/Air\ Quality\ Daily\ Summary.CSV /usr4/web/data.cmucreatelab.org/mirror-achd-air-quality-reports-esdr/tmp/tmp3SgC78


Exception: Call to subprocess_check failed with return code 6
Standard error:
Host key verification failed.
Standard out:
